# Task 1: E-commerce Data Processing


# Part A: Data Validation

In [1]:
orders = [
    {"customer": "Alice", "total": 250.5},
    {"customer": "Bob", "total": "invalid_data"},
    {"customer": "Charlie", "total": 450},
    {"customer": "Daisy", "total": 100.0},
    {"customer": "Eve", "total": -30},  # Invalid total
]

def validate_orders(orders):
    # Lambda function to filter out invalid orders
    is_valid_order = lambda order: isinstance(order["total"], (int, float)) and order["total"] > 0
    
    try:
        # Filter orders with proper error handling for conversion issues
        valid_orders = list(filter(is_valid_order, orders))
    except (ValueError, TypeError) as e:
        print(f"Error: {e}")
    
    return valid_orders

valid_orders = validate_orders(orders)
print("Valid Orders:", valid_orders)

Valid Orders: [{'customer': 'Alice', 'total': 250.5}, {'customer': 'Charlie', 'total': 450}, {'customer': 'Daisy', 'total': 100.0}]


# Part B: Discount Application

In [2]:
def apply_discount(orders):
    # Lambda function to apply 10% discount to orders above $300
    apply_discount_lambda = lambda order: {**order, "total": order["total"] * 0.9} if order["total"] > 300 else order
    
    # Using map to apply the discount
    discounted_orders = list(map(apply_discount_lambda, orders))
    
    return discounted_orders

discounted_orders = apply_discount(valid_orders)
print("Discounted Orders:", discounted_orders)

Discounted Orders: [{'customer': 'Alice', 'total': 250.5}, {'customer': 'Charlie', 'total': 405.0}, {'customer': 'Daisy', 'total': 100.0}]


# Part C: Total Sales Calculation

In [3]:
from functools import reduce

def calculate_total_sales(orders):
    # Lambda function to calculate total sales
    total_sales = reduce(lambda acc, order: acc + order["total"], orders, 0)
    
    return total_sales

total_sales = calculate_total_sales(discounted_orders)
print("Total Sales:", total_sales)

Total Sales: 755.5


# Task 2: Iterator and Generator


# Part A: Custom Iterator

In [6]:
class SquareIterator:
    def __init__(self, n):
        self.n = n
        self.current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current < self.n:
            self.current += 1
            return self.current ** 2
        else:
            raise StopIteration

square_iter = SquareIterator(20)
for square in square_iter:
    print(square)

1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400


# Part B: Fibonacci Generator

In [7]:
def fibonacci_generator(n):
    a, b = 0, 1
    while a <= n:
        yield a
        a, b = b, a + b

for num in fibonacci_generator(10):
    print(num)

0
1
1
2
3
5
8


# Task 3: Exception Handling and Function Decorator

# Part A: Chained Exceptions

In [16]:
class ZeroDivisorError(Exception):
    """Custom exception raised when divisor is zero."""
    pass

class NonNumericError(Exception):
    """Custom exception raised when a non-numeric input is encountered."""
    pass

def divide_numbers(numbers, divisor):
    try:
        # Raise custom exception if divisor is zero
        if divisor == 0:
            raise ZeroDivisorError("Cannot divide by zero")
        
        # Ensure all elements in 'numbers' are numeric
        for num in numbers:
            if not isinstance(num, (int, float)):
                raise NonNumericError(f"Non-numeric value encountered: {num}")
        
        # Perform division for valid numbers
        return [num / divisor for num in numbers]
    
    except ZeroDivisorError as zde:
        # Raise ZeroDivisorError if divisor is zero
        raise zde
    
    except NonNumericError as nne:
        # Raise NonNumericError if a non-numeric value is found in the list
        raise nne

    except Exception as e:
        # Handle any other exceptions (e.g., NameError if divisor is not defined)
        raise Exception("An error occurred during division") from e

# Testing Task 3A
try:
    # 'a' is not defined, it will raise a NameError
    print(divide_numbers([10, 20, "thirty"], 0))
except NameError:
    print("Error: Divisor 'a' is not defined. Please provide a valid divisor.")
except ZeroDivisorError as zde:
    print(f"ZeroDivisorError: {zde}")
except NonNumericError as nne:
    print(f"NonNumericError: {nne}")
except Exception as e:
    print(f"Exception: {e}")


ZeroDivisorError: Cannot divide by zero


# Part B: Exception Logging Decorator

In [18]:
import functools
import logging

# Configure logging to capture error-level messages
logging.basicConfig(level=logging.ERROR)

# Decorator to log exceptions during function execution
def log_exceptions(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            # Log the exception with details about function name and error type
            logging.error(f"Exception in {func.__name__}: {e.__class__.__name__} - {e}")
            raise  # Re-raise the exception after logging
    return wrapper

@log_exceptions
def risky_divide(x, y):
    """Divides x by y, but raises a ZeroDivisionError if y is 0."""
    return x / y

# Test risky_divide with error handling
try:
    # Attempt division by zero, which should trigger logging
    risky_divide(10, 0)
except ZeroDivisionError:
    pass  # Handled by the decorator, so no further action needed


ERROR:root:Exception in risky_divide: ZeroDivisionError - division by zero
